<a href="https://colab.research.google.com/github/Manikandans1610/AIML/blob/main/Youtude_video_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install youtube-transcript-api langchain faiss-cpu transformers sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 39.6 MB/s eta 0:00:00


In [ ]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
# youtube_summarizer_no_key.py

from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

# 1. Fetch Transcript
def get_transcript(video_url: str) -> str:
    try:
        video_id = video_url.split("v=")[-1].split("&")[0]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([entry["text"] for entry in transcript])
    except Exception as e:
        print(f"Transcript error: {e}")
        return ""

# 2. Chunk Transcript
def split_text(text: str):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return splitter.split_text(text)

# 3. Create Vector Store (FAISS + HuggingFace)
def create_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    return FAISS.from_texts(chunks, embedding=embeddings)

# 4. Get Most Relevant Chunks (top k)
def get_relevant_chunks(vectorstore, query: str = "What is this video about?", k=4):
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": k})
    docs = retriever.get_relevant_documents(query)
    return " ".join([doc.page_content for doc in docs])

# 5. Summarize using HuggingFace model
def summarize(text: str):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(text, max_length=200, min_length=50, do_sample=False)
    return summary[0]['summary_text']

# 6. Full Pipeline
def summarize_youtube_video(url: str):
    print("[+] Getting transcript...")
    transcript = get_transcript(url)
    if not transcript:
        return "Transcript unavailable."

    print("[+] Splitting text...")
    chunks = split_text(transcript)

    print("[+] Building vector store...")
    vectorstore = create_vector_store(chunks)

    print("[+] Retrieving relevant parts...")
    relevant_text = get_relevant_chunks(vectorstore)

    print("[+] Summarizing...")
    return summarize(relevant_text)

# 7. Run
if __name__ == "__main__":
    youtube_url = input("Enter YouTube video URL: ").strip()
    summary = summarize_youtube_video(youtube_url)
    print("\n📌 Summary:\n")
    print(summary)


Enter YouTube video URL: https://www.youtube.com/watch?v=c6xVXwF-ZmY
[+] Getting transcript...
[+] Splitting text...
[+] Building vector store...


<ipython-input-7-22805d6b89f0>:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



📌 Summary:

The world's most widely used digital map has just made a quiet update the one that's resonating deeply with millions in the philippines as of the 14th of april google maps now displays the west philippine sea as a maritime location which was previously shown as the south china sea the change is now live both on google maps as well as google earth. is being seen as a valuable contribution to what manila sees as the truthful representation of the region.
